In [11]:
import glob
import os
# import librosa
# from librosa import display
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
%matplotlib inline
plt.style.use('ggplot')
# import pydot
# import graphviz
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'Ubuntu'
plt.rcParams['font.monospace'] = 'Ubuntu Mono'
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 11
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['xtick.labelsize'] = 10
plt.rcParams['ytick.labelsize'] = 10
plt.rcParams['legend.fontsize'] = 11
plt.rcParams['figure.titlesize'] = 13

In [12]:
features = np.loadtxt('nn_simple_features.csv', delimiter=',')
labels = np.array(np.loadtxt('nn_simple_labels.csv', delimiter=','), dtype=np.int)

In [13]:
from sklearn.preprocessing import normalize

X_all = features
y_all = labels

In [14]:
from sklearn.model_selection import train_test_split
import time

# X_not_test, X_test, y_not_rest, y_test = train_test_split(
#     X_all, y_all, stratify=y_all, train_size=.85, random_state=round(time.time()))

X_train, X_test, y_train, y_test = train_test_split(
    X_all, y_all, stratify=y_all, train_size=.8, random_state=round(time.time()))

/home/hitmann/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [17]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(9460, 1280)
(9460, 5)
(2366, 1280)
(2366, 5)


In [24]:
## Imports
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import print_summary, plot_model

In [25]:
# Training parameters
batch_size = 32
num_classes = 5
epochs = 40
img_rows, img_cols = 40, 32
print(K.image_data_format())

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
# X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
print(X_train.shape)
print(y_train.shape)
# print(X_val.shape)
# print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

channels_last
(9460, 40, 32, 1)
(9460, 5)
(2366, 40, 32, 1)
(2366, 5)


In [26]:
from keras import regularizers
from keras.layers import BatchNormalization

def build_model():
    # Model definition
    model = Sequential()
    model.add(Conv2D(4, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(8, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(64, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(lr=.005),
                  metrics=['accuracy'])
    
    return model

In [27]:
model = build_model()

In [28]:
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 9460 samples, validate on 2366 samples
Epoch 1/40
9460/9460 [==============================] - 7s 692us/step - loss: 1.2430 - acc: 0.5079 - val_loss: 0.7306 - val_acc: 0.7287
Epoch 2/40
9460/9460 [==============================] - 5s 576us/step - loss: 0.8677 - acc: 0.6761 - val_loss: 0.6300 - val_acc: 0.7608
Epoch 3/40
9460/9460 [==============================] - 5s 581us/step - loss: 0.7314 - acc: 0.7236 - val_loss: 0.5014 - val_acc: 0.8204
Epoch 4/40
9460/9460 [==============================] - 5s 572us/step - loss: 0.6597 - acc: 0.7541 - val_loss: 0.4692 - val_acc: 0.8402
Epoch 5/40
9460/9460 [==============================] - 6s 618us/step - loss: 0.6160 - acc: 0.7732 - val_loss: 0.3628 - val_acc: 0.8762
Epoch 6/40
9460/9460 [==============================] - 6s 590us/step - loss: 0.5767 - acc: 0.7873 - val_loss: 0.3791 - val_acc: 0.8719
Epoch 7/40
9460/9460 [==============================] - 6s 593us/step - loss: 0.5675 - acc: 0.7953 - val_loss: 0.3434 - val_acc: 0.8846


In [29]:
score = model.evaluate(X_test, y_test, verbose=1)
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

2366/2366 [==============================] - 0s 162us/step

Test loss: 0.214748954758
Test accuracy: 0.921808960321


In [13]:
from keras.utils import print_summary, plot_model

print_summary(model)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 36, 28, 4)         104       
_________________________________________________________________
batch_normalization_1 (Batch (None, 36, 28, 4)         16        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 18, 14, 4)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 18, 14, 4)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 12, 8)         296       
_________________________________________________________________
batch_normalization_2 (Batch (None, 16, 12, 8)         32        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 6, 8)           0         
__________